In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('augmented.csv')
df.head()

,geohash6,day,timestamp,demand,lat,lat_err,long,long_err,h,m,dow,latlong_pca0,latlong_pca1,latlongerror_pca0,latlongerror_pca1,latlong_cluster,latlongerror_cluster,time_delta,time_delta_sin,hour_sin
0,212,18,20:0,0.020072,-5.353088,0.002747,90.653687,0.005493,20,0,4,45.348186,0.012163,-45.45809,-0.000014,3,1,4.833333,0.682671,0.250000
1,212,19,5:30,0.512506,-5.353088,0.002747,90.653687,0.005493,5,30,5,45.348186,0.012163,-45.45809,-0.000014,3,1,5.229167,0.509349,0.370590
2,212,20,12:15,0.668930,-5.353088,0.002747,90.653687,0.005493,12,15,6,45.348186,0.012163,-45.45809,-0.000014,3,1,6.510417,0.047507,1.000000
3,212,59,21:0,0.047361,-5.353088,0.002747,90.653687,0.005493,21,0,3,45.348186,0.012163,-45.45809,-0.000014,3,1,3.875000,0.971942,0.146447
4,212,60,10:45,1.000000,-5.353088,0.002747,90.653687,0.005493,10,45,4,45.348186,0.012163,-45.45809,-0.000014,3,1,4.447917,0.829673,0.933013


In [3]:
X = df[['geohash6', 'day', 'lat', 'lat_err', 'long', 'long_err', 'h', 'm',
       'dow', 'latlong_pca0', 'latlong_pca1', 'latlongerror_pca0', 'latlongerror_pca1',
       'latlong_cluster', 'latlongerror_cluster', 'time_delta', 'time_delta_sin',
       'hour_sin']]
Y = df['demand']

In [4]:
X.head()

,geohash6,day,lat,lat_err,long,long_err,h,m,dow,latlong_pca0,latlong_pca1,latlongerror_pca0,latlongerror_pca1,latlong_cluster,latlongerror_cluster,time_delta,time_delta_sin,hour_sin
0,212,18,-5.353088,0.002747,90.653687,0.005493,20,0,4,45.348186,0.012163,-45.45809,-0.000014,3,1,4.833333,0.682671,0.250000
1,212,19,-5.353088,0.002747,90.653687,0.005493,5,30,5,45.348186,0.012163,-45.45809,-0.000014,3,1,5.229167,0.509349,0.370590
2,212,20,-5.353088,0.002747,90.653687,0.005493,12,15,6,45.348186,0.012163,-45.45809,-0.000014,3,1,6.510417,0.047507,1.000000
3,212,59,-5.353088,0.002747,90.653687,0.005493,21,0,3,45.348186,0.012163,-45.45809,-0.000014,3,1,3.875000,0.971942,0.146447
4,212,60,-5.353088,0.002747,90.653687,0.005493,10,45,4,45.348186,0.012163,-45.45809,-0.000014,3,1,4.447917,0.829673,0.933013


In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4206321 entries, 0 to 4206320
Data columns (total 18 columns):
geohash6                int64
day                     int64
lat                     float64
lat_err                 float64
long                    float64
long_err                float64
h                       int64
m                       int64
dow                     int64
latlong_pca0            float64
latlong_pca1            float64
latlongerror_pca0       float64
latlongerror_pca1       float64
latlong_cluster         int64
latlongerror_cluster    int64
time_delta              float64
time_delta_sin          float64
hour_sin                float64
dtypes: float64(11), int64(7)
memory usage: 577.7 MB


In [6]:
params = {}
params['learning_rate'] = 0.01
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['sub_feature'] = 0.5
params['num_leaves'] = 100
params['min_data'] = 1000
params['min_hessian'] = 5
params['huber_delta'] = 0.5
params['lambda_l2'] = 0.0005

In [7]:
Y_test = np.zeros(len(Y))

In [11]:
X = X.values
Y = Y.values

In [13]:
for i, (train_ind, val_ind) in enumerate(KFold(n_splits=4, shuffle = True, 
                                               random_state=1989).split(X)):
    print('Training model %d' % i)
    d_train = lgb.Dataset(X[train_ind], label=Y[train_ind])
    d_valid = lgb.Dataset(X[val_ind], Y[val_ind])
    watchlist = [d_valid]
    
    clf = lgb.train(params, d_train, 5000, watchlist, verbose_eval = 100)
    
    Y_test += clf.predict(X)

Training model 0
[100]	valid_0's rmse: 0.104903
[200]	valid_0's rmse: 0.0854501
[300]	valid_0's rmse: 0.0770317
[400]	valid_0's rmse: 0.0722194
[500]	valid_0's rmse: 0.0686812
[600]	valid_0's rmse: 0.0656248
[700]	valid_0's rmse: 0.063335
[800]	valid_0's rmse: 0.0613056
[900]	valid_0's rmse: 0.0596808
[1000]	valid_0's rmse: 0.0583853
[1100]	valid_0's rmse: 0.0572803
[1200]	valid_0's rmse: 0.0563807
[1300]	valid_0's rmse: 0.0555417
[1400]	valid_0's rmse: 0.0547738
[1500]	valid_0's rmse: 0.0540788
[1600]	valid_0's rmse: 0.0534818
[1700]	valid_0's rmse: 0.0528988
[1800]	valid_0's rmse: 0.0524064
[1900]	valid_0's rmse: 0.0519425
[2000]	valid_0's rmse: 0.0515197
[2100]	valid_0's rmse: 0.051148
[2200]	valid_0's rmse: 0.0507777
[2300]	valid_0's rmse: 0.0504273
[2400]	valid_0's rmse: 0.0500834
[2500]	valid_0's rmse: 0.0497577
[2600]	valid_0's rmse: 0.0494956
[2700]	valid_0's rmse: 0.0492203
[2800]	valid_0's rmse: 0.0489326
[2900]	valid_0's rmse: 0.0486817
[3000]	valid_0's rmse: 0.0484449
[3100

In [14]:
Y_test /= 4

In [15]:
df_sub = pd.DataFrame({'demand': Y_test})

In [16]:
df_sub.to_csv('lgb.csv',index = False)